In [1]:
import pandas as pd;
from scipy.stats import ttest_ind

In [2]:
fall18 = pd.read_csv('Fall18Applicants.csv')

In [3]:
fall17 = pd.read_csv('PuGo7.csv')
fall17 = fall17[fall17.ApplYr == 2017]

In [4]:
print len(fall18)
print len(fall17)

22408
22347


In [5]:
toDrop = ['AcademicQtrKeyId', u'ApplQtr', u'FiscalYr', u'SDBSrcSystemKey', u'StudentName',
          u'FAFSAStudentBudget', u'FAFSATuition', u'TuitionRemainingNoNeg', u'TuitionRemaining', 
          u'TotalBudgetRemainingNoNeg', u'TotalBudgetRemaining', u'Major_Assign1', u'Equity', 
          u'ParentalMaritalStatus', u'appl_rcvd_dt', u'UnfundedNeedNoNeg', u'UnfundedNeed',  
          u'WorkAndAllLoans', u'Total_Budget']
epsColumns = ['UWAttraction_TotalCount',
 'UWAttraction_TotalPct',
 'UWAttraction_ProdPct',
 'UWLocationAttraction_TotalCount',
 'UWLocationAttraction_TotalPct',
 'UWEdGoals_TotalCount',
 'UWEdGoals_TotalPct',
 'UWEdGoals_ProdPct',
 'LowerIncomePct',
 'LowerIncomeCount',
 'HigherIncomeCount',
 'LowerParentEdPct',
 'LowerParentEdCount',
 'HigherParentEdCount',
 'CollegeLocationPct_BorderState',
 'FamilyIncomePct_140to200k',
 'FirstGenerationPct_Not1stGen',
 'ParentEducationPct_NoResponse']
epsDerived = ['DeltaParentEdMinusAcademic', 'DeltaAcademicMinusParentEd', 'DeltaIncomeMinusAcademic', 'DeltaAcademicMinusIncome']
fall17 = fall17.drop(toDrop, axis = 1)
fall17 = fall17.drop(epsColumns, axis = 1)
fall17 = fall17.drop(epsDerived, axis = 1)

In [6]:
fall18.columns

Index([u'AcademicQtrKeyId', u'ApplQtr', u'ApplYr', u'FiscalYr',
       u'SDBSrcSystemKey', u'HolisticAcademicScore', u'HolisticPQAScore',
       u'HolisticAdmissionScore', u'Parent1_Ed_Level', u'Parent2_Ed_Level',
       u'HighestParentalEducation', u'TestScore', u'AcademicScore',
       u'AcademicRank', u'HS_AI', u'HS_GPA', u'HighSchoolCode', u'HS_State',
       u'AgeDecimal', u'AgeYrs', u'FirstGeneration_4YrDegree',
       u'FirstGeneration_Matriculated', u'Athlete', u'CollegeInTheHighSchool',
       u'RunningStart', u'TR_GPA', u'EntryClass', u'AlumOffspring',
       u'Conditional', u'Provisional', u'Major_Req1', u'Old_Major_Category',
       u'CIP', u'NaturalResources', u'Architecture', u'EthnicCultStudies',
       u'Communications', u'Education', u'EngineeringCompSci', u'ForeignLang',
       u'SocialSciences', u'Biology', u'Math', u'HistPhil', u'Psych', u'Arts',
       u'Health', u'Business', u'Dependency', u'FAFSAFiledOnTime', u'EFC',
       u'Income', u'GrantSchol', u'NonPLUSLoan

In [7]:
fall17.columns

Index([u'ApplYr', u'IsEnrolled', u'IsAdmitted', u'PuGoOffer',
       u'HolisticAcademicScore', u'HolisticPQAScore',
       u'HolisticAdmissionScore', u'parent1_ed_level', u'parent2_ed_level',
       u'HighestParentalEducation', u'TestScore', u'AcademicScore',
       u'AcademicRank', u'HS_AI', u'HighSchoolGPA', u'HighSchoolCode',
       u'HS_For_Lang_Yrs', u'HS_Math_Level', u'HS_Yrs_Arts',
       u'HS_Yrs_For_Lang', u'HS_Yrs_Math', u'HS_Yrs_Science',
       u'HS_Yrs_Soc_Sci', u'HS_State', u'AgeDecimal', u'AgeYrs',
       u'FirstGeneration_4YrDegree', u'FirstGeneration_Matriculated',
       u'Athlete', u'CollegeInTheHighSchool', u'RunningStart', u'TR_GPA',
       u'EntryClass', u'AlumOffspring', u'Conditional', u'Provisional',
       u'Major_Req1', u'Major_Category', u'CIP', u'NaturalResources',
       u'Architecture', u'EthnicCultStudies', u'Communications', u'Education',
       u'EngineeringCompSci', u'ForeignLang', u'SocialSciences', u'Biology',
       u'Math', u'HistPhil', u'Psych', 

In [8]:
print [x for x in fall18.columns if x not in fall17.columns and x not in toDrop]

['Parent1_Ed_Level', 'Parent2_Ed_Level', 'HS_GPA', 'Old_Major_Category']


In [9]:
toIgnore = ['ApplYr', 'IsAdmitted', 'IsEnrolled', 'PuGoOffer', 'Tuition']
print [x for x in fall17.columns if x not in fall18.columns and x not in toIgnore]

['parent1_ed_level', 'parent2_ed_level', 'HighSchoolGPA', 'Major_Category']


In [10]:
fall18['HighSchoolGPA'] = fall18.HS_GPA
fall18['parent1_ed_level'] = fall18.Parent1_Ed_Level
fall18['parent2_ed_level'] = fall18.Parent2_Ed_Level
fall18['Major_Category'] = fall18.Old_Major_Category

fall18 = fall18.drop(['Parent1_Ed_Level', 'Parent2_Ed_Level', 'HS_GPA', 'Old_Major_Category'], axis = 1)

In [11]:
stringCols = ['HS_State', 'Major_Req1', 'FINGroupings']
colsToCheck = [x for x in fall17.columns if x not in toIgnore and x not in stringCols]

In [12]:
yesNoCols = ['FirstGeneration_4YrDegree', 'FirstGeneration_Matriculated', 
             'CollegeInTheHighSchool', 'RunningStart', 'Major_Category', 'Dependency', 'FAFSAFiledOnTime']
stringSwitch = {'Dependency': 'Dependent'}
for each in yesNoCols:
    check = 'Y'
    if each in stringSwitch.keys():
        check = stringSwitch[each]
    fall17[each] = fall17[each].str.strip() == check
    fall18[each] = fall18[each].str.strip() == check

In [13]:
pValues = []
for each in colsToCheck:
    prior = fall17[each]
    current = fall18[each]
    pValues.append(ttest_ind(prior, current)[1])
store = pd.DataFrame(index = colsToCheck)
store['mean17'] = fall17[colsToCheck].mean()
store['mean18'] = fall18[colsToCheck].mean()
store['pValues'] = pd.Series(pValues, index = colsToCheck)

C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\stats\stats.py:3851: RuntimeWarning: invalid value encountered in absolute
  prob = distributions.t.sf(np.abs(t), df) * 2  # use np.abs to get upper tail
C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [14]:
store

mean17         mean18        pValues
HolisticAcademicScore             11.457601      11.351214   8.955518e-03
HolisticPQAScore                   7.084083       7.001607   8.354297e-04
HolisticAdmissionScore            11.494653      11.377588   3.963565e-03
parent1_ed_level                   4.806062       4.856658            NaN
parent2_ed_level                   4.592375       4.724571            NaN
HighestParentalEducation           5.138230       5.218497            NaN
TestScore                       1291.324079    1310.323399            NaN
AcademicScore                      0.860817       0.850180   5.906937e-13
AcademicRank                    2836.937620    2493.520395  6.782400e-201
HS_AI                             68.643032      71.133378            NaN
HighSchoolGPA                      3.591231       3.627880            NaN
HighSchoolCode                176573.760504  188827.803441            NaN
HS_For_Lang_Yrs                    3.457442       3.347064            NaN
HS_Math_Level                      4.595990       4.390084            NaN
HS_Yrs_Arts                        1.616397       0.000000            NaN
HS_Yrs_For_Lang                    2.505102       0.000000            NaN
HS_Yrs_Math                        3.450595       0.000000            NaN
HS_Yrs_Science                     3.264656       0.000000            NaN
HS_Yrs_Soc_Sci                     2.768997       0.000000            NaN
AgeDecimal                        18.515969      18.509676   1.736814e-01
AgeYrs                            18.020316      18.009193   2.443145e-02
FirstGeneration_4YrDegree          0.177518       0.156283   1.691945e-09
FirstGeneration_Matriculated       0.098044       0.075107   6.036928e-18
Athlete                            0.003177       0.004240   6.451307e-02
CollegeInTheHighSchool             0.156844       0.230275   2.168459e-86
RunningStart                       0.022732       0.020439   9.508191e-02
TR_GPA                             3.728858       3.764058            NaN
EntryClass                         1.000000       0.999866   1.803051e-01
AlumOffspring                      0.029803       0.022001   2.031761e-07
Conditional                        0.006668       0.004864   1.275575e-02
Provisional                        0.046986       0.143342  2.330240e-127
Major_Category                     0.000000       0.000000            NaN
CIP                               27.511559      29.897760            NaN
NaturalResources                   0.026804       0.019770   8.013987e-07
Architecture                       0.011903       0.010309   1.075528e-01
EthnicCultStudies                  0.020226       0.016646   4.867166e-03
Communications                     0.051237       0.024232   6.090225e-51
Education                          0.049358       0.010309  3.627759e-131
EngineeringCompSci                 0.248221       0.238933   2.207808e-02
ForeignLang                        0.008771       0.010443   6.983579e-02
SocialSciences                     0.054056       0.066271   5.516481e-08
Biology                            0.167405       0.144770   4.149261e-11
Math                               0.000000       0.015753   1.862346e-79
HistPhil                           0.010561       0.011112   5.732104e-01
Psych                              0.047613       0.044805   1.572377e-01
Arts                               0.011500       0.022715   5.690985e-20
Health                             0.087976       0.100455   6.219320e-06
Business                           0.109769       0.111523   5.542287e-01
Dependency                         0.678838       0.639191   8.733949e-19
FAFSAFiledOnTime                   0.591802       0.598849   1.288717e-01
EFC                            59038.121548   64559.385019            NaN
Income                        196616.640381  205061.983831            NaN
GrantSchol                       315.355887       0.000000  5.234629e-196
NonPLUSLoans                    1709.3251

In [20]:
fall17.HS_Yrs_Arts.describe()

count    22346.000000
mean         1.616397
std          1.583391
min          0.000000
25%          0.000000
50%          1.000000
75%          3.000000
max          9.000000
Name: HS_Yrs_Arts, dtype: float64

In [21]:
fall18.HS_Yrs_Arts.describe()

count    22408.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: HS_Yrs_Arts, dtype: float64